# Práctica 2 segunda parte
## Reimplementación con Cython para MaxFlowAeiu

El objetivo consiste en reimplementar nuestro método numérico realizado en la práctica 1 con niveles de BLAS, cómputo en paralelo (CPU/GPU), con compilación a C (por ejemplo vía cython, rcpp) o julia guiándose del perfilamiento de memoria, uso de procesador o tiempo de ejecución de su paquete. 
Usando una máquina de AWS con las siguientes características:

## Características de la instancia

La máquina `m4.16xlarge` tiene las siguientes características:

In [1]:
%%bash
lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit
Byte Order:                      Little Endian
CPU(s):                          4
On-line CPU(s) list:             0-3
Thread(s) per core:              1
Core(s) per socket:              4
Socket(s):                       1
Vendor ID:                       0x00
Model:                           0
Stepping:                        0x0
BogoMIPS:                        48.00
Vulnerability Itlb multihit:     Not affected
Vulnerability L1tf:              Not affected
Vulnerability Mds:               Not affected
Vulnerability Meltdown:          Not affected
Vulnerability Spec store bypass: Vulnerable
Vulnerability Spectre v1:        Mitigation; __user pointer sanitization
Vulnerability Spectre v2:        Not affected
Vulnerability Srbds:             Not affected
Vulnerability Tsx async abort:   Not affected
Flags:                           fp asimd evtstrm aes pmull sha1 sha2 crc32 atomics fphp asimdhp cpuid asimd

In [2]:
%%bash
uname -ar #r for kernel, a for all

Linux cb8cadb26507 5.10.47-linuxkit #1 SMP PREEMPT Sat Jul 3 21:50:16 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux


## Cython
Es un compilador que traduce instrucciones *anotadas* y escritas en un lenguaje híbrido entre Python y C que resultan un módulo compilado. Este módulo puede ser importado como un módulo regular de Python utilizando import. Típicamente el módulo compilado resulta ser similar en sintaxis al lenguaje C.

## Primera iteración

Para utilizar Cython necesitamos de 3 archivos:

1.El código que será compilado en un archivo con extensión .pyx (escrito en Python)
    
2.Un archivo setup.py que contiene las instrucciones para llamar a Cython y se encarga de crear el módulo compilado.
  
3.El código escrito en Python que importará el módulo compilado.

### Creación del archivo con extensión .pyx
Vamos a copiar nuestro código del método MaxFlowAeiu

In [3]:
%%file script/mfaeiu1.pyx

from collections import defaultdict

class MaxFlowAeiu:
    '''
        Finds the paths in order to return the maximum flow in the
        network. This class implements the Ford Fulkerson method.
    '''

    def __init__(self,graph):
        '''
        Initialices parts of the problem.
        Attributes:
            graph (matrix): defines the graph from a matrix,
            N (bool): the number of the nodes in the graph,
            source (int): index the source node of the graph,
            sink (int): index the sink node of the graph.
            residualgraph(matrix):graph where the residual values 
                                  of the edges are updated after 
                                  each iteration.
        '''
        self.graph = graph.copy()
        self.N = len(graph)
        self.source = 0      
        self.sink= self.N-1
        self.residualgraph = self.graph.copy()
    
    def busq_anchura(self,source,sink,parent): 
        '''
        Defines queue of the visited nodes and the parents of them, and
        so long as the queue of the nodes that need to be visited is not empty, the algorithm goes on.
        
        Args:
            source (int): index the source node of the graph.
            sink (int): index the sink node of the graph.
            parent (list): vector for keeping track of the parents of visited nodes.
        Attributes:
            visit (list): vector for keeping track of visited nodes,
            queue (list): the queue of the nodes needed to be visited,
            parent (list): vector for keeping track of the parents of visited nodes.
        Returns:
            (bool): A True/False indicating the presence or absence of a path. 
            It also updates the parent list with the information necessary to reconstruct the path.
  
        '''
        # se inicia el vector de visit en False de acuerdo al nuemero de nodos 
        visit=[False]*(self.N)
        # se inicia el vector queue vacio     
        queue=[]    

        # se agerga al vector queue el valor  de source y 
        # mediante source se asigna al primer espacio del vector visit en true 
        #para comenzar nuestra busqueda del path
        queue.append(source)              
        visit[source]=True               

        while queue:

            # se extrae siempre el primer valor del queue y seb asigna a u
            u = queue.pop(0)
            
            # Se requiere tanto el index como el valor del nodo que sera padre
            for index, value in enumerate(self.graph[u]): 
                if visit[index]== False and value > 0:
                    queue.append(index)
                    visit[index] = True
                    parent[index]=u
        
        # Se crea un check ternario para regresar True 
        # si el camino sido recorrido completamente False de lo contrario
        return True if visit[sink] else False


    # metodo para ejecutar algoritmo de ford fulkerson
    def ford_fulkerson(self):
        '''
        Runs the Ford Fulkerson method, keeping and updating the residual graph 
        and running over the bfs function in all the nodes.
        Args:
            source (int): index the source node of the graph.
            sink (int): index the sink node of the graph.
        Attributes:
            residualgraph (matrix):matrix the residual graph,
            path_flow (float): we need to calculate the min flow of the selected path,
            parent: vector for keeping track of the parents of visited nodes.
        Returns:
            maximum_flow (float): calculated maximum flow of the graph.
            
        '''
        # se inicia parent en -1 de acuerdo a la cantidad de nodos
        parent=[-1]*(self.N)

        # se inicia maximun_flow en 0
        maximum_flow=0
            
        while self.busq_anchura(self.source,self.sink,parent):
            # se inicia pathflow en inf float 
            # que contendra el minimo flujo del path seleccionado
            path_flow = float('inf')     
            j = self.sink                    
                
            while not j == self.source:
                # se calcula el minimo de todo el path 
                path_flow=min(path_flow, self.residualgraph[parent[j]][j])
                # se asigna el valor del nodo padre  
                j=parent[j]

            
            # se actualiza los valores residuales de los edges en self.residualgraph
            v = self.sink
            while not v == self.source:
                u=parent[v]
                self.residualgraph[u][v] -= path_flow                   
                self.residualgraph[v][u] += path_flow
                v=parent[v]


            # se agrega el path_flow para calcular el maximo
            maximum_flow += path_flow  
            
        return maximum_flow
    
    # metodo para obtener información del grafo
    def infoMF(self):
        '''
        Obtains the information of the graph, regarding to the number of nodes and arcs
        Args:
            grapf (matrix): defines the graph from a matrix.
        Returns:
            Printed info of the graph.
            
        '''
        # Función para contar nodos y ramas
        rama = 0
        arreglo = self.graph
        for i in range(len(arreglo)):
            for m in range(len(arreglo[i])):
                if arreglo[i][m] == 0:
                    pass
                else:
                    rama = rama + 1
        print("nodos",len(arreglo))
        print("ramas",rama)

Overwriting script/mfaeiu1.pyx


### Creación del archivo setup.py
Archivo setup.py que contiene las instrucciones para el build:

In [4]:
%%file script/setup_1.py
from distutils.core import setup
from Cython.Build import cythonize

setup(ext_modules = cythonize("script/mfaeiu1.pyx", 
                              compiler_directives={'language_level' : 3}))

Overwriting script/setup_1.py


### Compilación desde línea de comandos

In [5]:
%%bash
python3 script/setup_1.py build_ext --inplace

Compiling script/mfaeiu1.pyx because it changed.
[1/1] Cythonizing script/mfaeiu1.pyx
running build_ext
building 'mfaeiu1' extension
creating build
creating build/temp.linux-x86_64-3.8
creating build/temp.linux-x86_64-3.8/script
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.8 -c script/mfaeiu1.c -o build/temp.linux-x86_64-3.8/script/mfaeiu1.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.8/script/mfaeiu1.o -o /datos/practica-2-segunda-parte-EddOselotl/notebooks_apoyo/mfaeiu1.cpython-38-x86_64-linux-

### Prueba 1
Probamos la función .pyx con la base de siempre.

In [6]:
import time
import networkx as nx
from pytest import approx
import pandas as pd
import numpy as np
from IPython.display import HTML, display

In [7]:
from mfaeiu1 import MaxFlowAeiu

In [8]:
start_time = time.time()

url_d = "https://raw.githubusercontent.com/optimizacion-2-2022-gh-classroom/practica-2-primera-parte-urieluard/main/BD/d.csv"
d = pd.read_csv(url_d,header=None)
arreglo = d.values.tolist()
MF=MaxFlowAeiu(arreglo)
print("The maximum flow in this network is {}".format(MF.ford_fulkerson()))

end_time = time.time()

The maximum flow in this network is 1480.0


In [9]:
secs_1 = end_time-start_time
print("El algoritmo tomó",secs_1,"segundos" )

El algoritmo tomó 0.422684907913208 segundos


Y verificamos que continuamos resolviendo el problema de manera correcta.

In [10]:
# Scipy
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow


Para poder usar la función de flujo máximo de `scipy`, es necesario tener la matriz en formato _sparse_, una vez representada de esta manera, es sencillo encontrar el valor del fliujo máximo.

In [11]:
# Generamos el arreglo final de tipo "numpy array"
arreglo = d.to_numpy()
arreglo
arreglo2=arreglo.astype(int)
graph = csr_matrix(arreglo2)
maximum_flow(graph, 0, 43).flow_value

1480

In [12]:
arreglo = d.values.tolist()
MF=MaxFlowAeiu(arreglo)
mfaeiu=MF.ford_fulkerson()

In [13]:
flow_val=maximum_flow(graph, 0, 43).flow_value
print(flow_val == mfaeiu)

True


### Archivo HTML
Vía línea de comando

In [15]:
%%bash
# Para usar en Binder
#/srv/conda/envs/notebook/bin/cython --force -3 --annotate script/mfaeiu1.pyx
# Para usar en servidor local
$HOME/.local/bin/cython --force -3 --annotate script/mfaeiu1.pyx

In [16]:
display(HTML("script/mfaeiu1.html"))

## Segunda Iteración

### Creación del archivo con extensión .pyx
Vamos a intentar mejorar las líneas que se ven *muy amarillas* de lo que vimos en la celda anterior, tendríamos que hacer modificaciones al `for` en la línea 63 y al update del grafo residual en las líneas 113 y 114. 

In [17]:
%%file script/mfaeiu2.pyx

from collections import defaultdict

class MaxFlowAeiu:
    '''
        Finds the paths in order to return the maximum flow in the
        network. This class implements the Ford Fulkerson method.
    '''

    def __init__(self,graph):
        '''
        Initialices parts of the problem.
        Attributes:
            graph (matrix): defines the graph from a matrix,
            N (bool): the number of the nodes in the graph,
            source (int): index the source node of the graph,
            sink (int): index the sink node of the graph.
            residualgraph(matrix):graph where the residual values 
                                  of the edges are updated after 
                                  each iteration.
        '''
        self.graph = graph.copy()
        self.N = len(graph)
        self.source = 0      
        self.sink= self.N-1
        self.residualgraph = self.graph.copy()
    
    def busq_anchura(self,unsigned int source,unsigned int sink,parent): 
        '''
        Defines queue of the visited nodes and the parents of them, and
        so long as the queue of the nodes that need to be visited is not empty, the algorithm goes on.
        
        Args:
            source (int): index the source node of the graph.
            sink (int): index the sink node of the graph.
            parent (list): vector for keeping track of the parents of visited nodes.
        Attributes:
            visit (list): vector for keeping track of visited nodes,
            queue (list): the queue of the nodes needed to be visited,
            parent (list): vector for keeping track of the parents of visited nodes.
        Returns:
            (bool): A True/False indicating the presence or absence of a path. 
            It also updates the parent list with the information necessary to reconstruct the path.
  
        '''
        # se inicia el vector de visit en False de acuerdo al nuemero de nodos 
        visit=[False]*(self.N)
        # se inicia el vector queue vacio     
        queue=[]    

        # se agerga al vector queue el valor  de source y 
        # mediante source se asigna al primer espacio del vector visit en true 
        #para comenzar nuestra busqueda del path
        queue.append(source)              
        visit[source]=True               

        while queue:

            # se extrae siempre el primer valor del queue y seb asigna a u
            u = queue.pop(0)
            
            # Se requiere tanto el index como el valor del nodo que sera padre
            for index, value in enumerate(self.graph[u]): 
                if visit[index]== False and value > 0:
                    queue.append(index)
                    visit[index] = True
                    parent[index]=u
        
        # Se crea un check ternario para regresar True 
        # si el camino sido recorrido completamente False de lo contrario
        return True if visit[sink] else False


    # metodo para ejecutar algoritmo de ford fulkerson
    def ford_fulkerson(self):
        '''
        Runs the Ford Fulkerson method, keeping and updating the residual graph 
        and running over the bfs function in all the nodes.
        Args:
            source (int): index the source node of the graph.
            sink (int): index the sink node of the graph.
        Attributes:
            residualgraph (matrix):matrix the residual graph,
            path_flow (float): we need to calculate the min flow of the selected path,
            parent: vector for keeping track of the parents of visited nodes.
        Returns:
            maximum_flow (float): calculated maximum flow of the graph.
            
        '''
        # se inicia parent en -1 de acuerdo a la cantidad de nodos
        parent=[-1]*(self.N)

        # se inicia maximun_flow en 0
        maximum_flow=0
            
        while self.busq_anchura(self.source,self.sink,parent):
            # se inicia pathflow en inf float 
            # que contendra el minimo flujo del path seleccionado
            path_flow = float('inf')     
            j = self.sink                    
                
            while not j == self.source:
                # se calcula el minimo de todo el path 
                path_flow=min(path_flow, self.residualgraph[parent[j]][j])
                # se asigna el valor del nodo padre  
                j=parent[j]

            
            # se actualiza los valores residuales de los edges en self.residualgraph
            v = self.sink
            while not v == self.source:
                u=parent[v]
                self.residualgraph[u][v] -= path_flow                   
                self.residualgraph[v][u] += path_flow
                v=parent[v]


            # se agrega el path_flow para calcular el maximo
            maximum_flow += path_flow  
            
        return maximum_flow
    
    # metodo para obtener información del grafo
    def infoMF(self):
        '''
        Obtains the information of the graph, regarding to the number of nodes and arcs
        Args:
            grapf (matrix): defines the graph from a matrix.
        Returns:
            Printed info of the graph.
            
        '''
        # Función para contar nodos y ramas
        rama = 0
        arreglo = self.graph
        for i in range(len(arreglo)):
            for m in range(len(arreglo[i])):
                if arreglo[i][m] == 0:
                    pass
                else:
                    rama = rama + 1
        print("nodos",len(arreglo))
        print("ramas",rama)

Overwriting script/mfaeiu2.pyx


### Archivo HTML
Vía línea de comando

In [19]:
%%bash
# Para usar en Binder
#/srv/conda/envs/notebook/bin/cython --force -3 --annotate script/mfaeiu2.pyx
# Para usar en servidor local
$HOME/.local/bin/cython --force -3 --annotate script/mfaeiu2.pyx 

In [20]:
display(HTML("script/mfaeiu2.html"))

### Probamos la reimplementación

In [21]:
%%file script/setup_2.py
from distutils.core import setup
from Cython.Build import cythonize

setup(ext_modules = cythonize("script/mfaeiu2.pyx", 
                              compiler_directives={'language_level' : 3}))

Overwriting script/setup_2.py


### Compilación desde línea de comandos

In [22]:
%%bash
python3 script/setup_2.py build_ext --inplace

running build_ext
building 'mfaeiu2' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.8 -c script/mfaeiu2.c -o build/temp.linux-x86_64-3.8/script/mfaeiu2.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.8/script/mfaeiu2.o -o /datos/practica-2-segunda-parte-EddOselotl/notebooks_apoyo/mfaeiu2.cpython-38-x86_64-linux-gnu.so


In [23]:
from mfaeiu2 import MaxFlowAeiu

In [24]:
start_time = time.time()

url_d = "https://raw.githubusercontent.com/optimizacion-2-2022-gh-classroom/practica-2-primera-parte-urieluard/main/BD/d.csv"
d = pd.read_csv(url_d,header=None)
arreglo = d.values.tolist()
MF=MaxFlowAeiu(arreglo)
print("The maximum flow in this network is {}".format(MF.ford_fulkerson()))

end_time = time.time()

The maximum flow in this network is 1480.0


In [25]:
secs_2 = end_time-start_time
print("El algoritmo tomó",secs_2,"segundos" )

El algoritmo tomó 0.3366208076477051 segundos


In [26]:
print("Redujimos el tiempo en: {} segundos".format(secs_1 - secs_2))

Redujimos el tiempo en: 0.08606410026550293 segundos


## Tercera Iteración

### Creación del archivo .pyx
Seguimos intentando

In [27]:
%%file script/mfaeiu3.pyx

from collections import defaultdict

class MaxFlowAeiu:
    '''
        Finds the paths in order to return the maximum flow in the
        network. This class implements the Ford Fulkerson method.
    '''

    def __init__(self,graph):
        '''
        Initialices parts of the problem.
        Attributes:
            graph (matrix): defines the graph from a matrix,
            N (bool): the number of the nodes in the graph,
            source (int): index the source node of the graph,
            sink (int): index the sink node of the graph.
            residualgraph(matrix):graph where the residual values 
                                  of the edges are updated after 
                                  each iteration.
        '''
        self.graph = graph.copy()
        self.N = len(graph)
        self.source = 0      
        self.sink= self.N-1
        self.residualgraph = self.graph.copy()
    
    def busq_anchura(self,unsigned int source,unsigned int sink,parent): 
        '''
        Defines queue of the visited nodes and the parents of them, and
        so long as the queue of the nodes that need to be visited is not empty, the algorithm goes on.
        
        Args:
            source (int): index the source node of the graph.
            sink (int): index the sink node of the graph.
            parent (list): vector for keeping track of the parents of visited nodes.
        Attributes:
            visit (list): vector for keeping track of visited nodes,
            queue (list): the queue of the nodes needed to be visited,
            parent (list): vector for keeping track of the parents of visited nodes.
        Returns:
            (bool): A True/False indicating the presence or absence of a path. 
            It also updates the parent list with the information necessary to reconstruct the path.
  
        '''
        # se inicia el vector de visit en False de acuerdo al nuemero de nodos 
        visit=[False]*(self.N)
        # se inicia el vector queue vacio     
        queue=[]    

        # se agerga al vector queue el valor  de source y 
        # mediante source se asigna al primer espacio del vector visit en true 
        #para comenzar nuestra busqueda del path
        queue.append(source)              
        visit[source]=True               

        while queue:

            # se extrae siempre el primer valor del queue y seb asigna a u
            u = queue.pop(0)
            
            # Se requiere tanto el index como el valor del nodo que sera padre
            for index, value in enumerate(self.graph[u]): 
                if visit[index]== False and value > 0:
                    queue.append(index)
                    visit[index] = True
                    parent[index]=u
        
        # Se crea un check ternario para regresar True 
        # si el camino sido recorrido completamente False de lo contrario
        return True if visit[sink] else False


    # metodo para ejecutar algoritmo de ford fulkerson
    def ford_fulkerson(self):
        '''
        Runs the Ford Fulkerson method, keeping and updating the residual graph 
        and running over the bfs function in all the nodes.
        Args:
            source (int): index the source node of the graph.
            sink (int): index the sink node of the graph.
        Attributes:
            residualgraph (matrix):matrix the residual graph,
            path_flow (float): we need to calculate the min flow of the selected path,
            parent: vector for keeping track of the parents of visited nodes.
        Returns:
            maximum_flow (float): calculated maximum flow of the graph.
            
        '''
        # se inicia parent en -1 de acuerdo a la cantidad de nodos
        parent=[-1]*(self.N)

        # se inicia maximun_flow en 0
        maximum_flow=0
            
        while self.busq_anchura(self.source,self.sink,parent):
            # se inicia pathflow en inf float 
            # que contendra el minimo flujo del path seleccionado
            path_flow = float('inf')     
            j = self.sink                    
                
            while not j == self.source:
                # se calcula el minimo de todo el path 
                path_flow=min(path_flow, self.residualgraph[parent[j]][j])
                # se asigna el valor del nodo padre  
                j=parent[j]

            
            # se actualiza los valores residuales de los edges en self.residualgraph
            v = self.sink
            while not v == self.source:
                u=parent[v]
                self.residualgraph[u][v] -= path_flow                   
                self.residualgraph[v][u] += path_flow
                v=parent[v]


            # se agrega el path_flow para calcular el maximo
            maximum_flow += path_flow  
            
        return maximum_flow
    
    # metodo para obtener información del grafo
    def infoMF(self):
        '''
        Obtains the information of the graph, regarding to the number of nodes and arcs
        Args:
            grapf (matrix): defines the graph from a matrix.
        Returns:
            Printed info of the graph.
            
        '''
        # Función para contar nodos y ramas
        rama = 0
        arreglo = self.graph
        for i in range(len(arreglo)):
            for m in range(len(arreglo[i])):
                if arreglo[i][m] == 0:
                    pass
                else:
                    rama = rama + 1
        print("nodos",len(arreglo))
        print("ramas",rama)

Overwriting script/mfaeiu3.pyx


### Archivo HTML

In [28]:
%%bash
# Para usar en Binder
#/srv/conda/envs/notebook/bin/cython --force -3 --annotate script/mfaeiu3.pyx 

# Para usar en servidor local
$HOME/.local/bin/cython --force -3 --annotate script/mfaeiu3.pyx 

In [29]:
display(HTML("script/mfaeiu3.html"))

### Prueba de la reimplementación

In [30]:
%%file script/setup_3.py
from distutils.core import setup
from Cython.Build import cythonize

setup(ext_modules = cythonize("script/mfaeiu3.pyx", 
                              compiler_directives={'language_level' : 3}))

Overwriting script/setup_3.py


### Compilación desde línea de comandos

In [31]:
%%bash
python3 script/setup_3.py build_ext --inplace

running build_ext
building 'mfaeiu3' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.8 -c script/mfaeiu3.c -o build/temp.linux-x86_64-3.8/script/mfaeiu3.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.8/script/mfaeiu3.o -o /datos/practica-2-segunda-parte-EddOselotl/notebooks_apoyo/mfaeiu3.cpython-38-x86_64-linux-gnu.so


In [32]:
from mfaeiu3 import MaxFlowAeiu

In [33]:
start_time = time.time()

url_d = "https://raw.githubusercontent.com/optimizacion-2-2022-gh-classroom/practica-2-primera-parte-urieluard/main/BD/d.csv"
d = pd.read_csv(url_d,header=None)
arreglo = d.values.tolist()
MF=MaxFlowAeiu(arreglo)
print("The maximum flow in this network is {}".format(MF.ford_fulkerson()))

end_time = time.time()

The maximum flow in this network is 1480.0


In [34]:
secs_3 = end_time-start_time
print("El algoritmo tomó",secs_3,"segundos" )

El algoritmo tomó 0.25727128982543945 segundos


In [35]:
print("Redujimos el tiempo en: {} segundos".format(secs_2 - secs_3))

Redujimos el tiempo en: 0.07934951782226562 segundos


In [36]:
print("Redujimos el tiempo en: {} segundos con respecto al original".format(secs_1 - secs_3))
print("Redujimos el tiempo en: {} segundos con respecto a la primera reimplementación".format(secs_2 - secs_3))

Redujimos el tiempo en: 0.16541361808776855 segundos con respecto al original
Redujimos el tiempo en: 0.07934951782226562 segundos con respecto a la primera reimplementación


In [37]:
print("La reimplementación es {} veces más rápida que nuestro código original compilado a C.".format(secs_1 / secs_3))

La reimplementación es 1.6429540513440228 veces más rápida que nuestro código original compilado a C.


# Cuarta iteración

Aqui hacemos uso de los comandos magic para cython

In [38]:
%load_ext Cython

In [39]:
%%cython --annotate

import numpy as np
from cpython cimport array
import array


cdef class MaxFlowAeiu:
    '''
        Finds the paths in order to return the maximum flow in the
        network. This class implements the Ford Fulkerson method.
    '''
    cdef unsigned int N, source, sink, maximum_flow
    cdef int[:,:] graph

    def __init__(self,list graph):
        '''
        Initialices parts of the problem.
        Attributes:
            graph (array 2D): defines the graph from a matrix,
            N (bool): the number of the nodes in the graph,
            source (int): index the source node of the graph,
            sink (int): index the sink node of the graph.
            residualgraph(matrix):graph where the residual values 
                                  of the edges are updated after 
                                  each iteration.
        '''

        # se asigna dtype para numpy array
        DTYPE = np.intc
        

        self.graph = np.array(graph,dtype=DTYPE)
        self.N = len(self.graph)
        self.source = 0      
        self.sink= self.N-1
        self.maximum_flow =0
    
    def busq_anchura(self,unsigned int source,unsigned int sink,signed int[:] parent): 
        '''
        Defines queue of the visited nodes and the parents of them, and
        so long as the queue of the nodes that need to be visited is not empty, the algorithm goes on.
        
        Args:
            source (int): index the source node of the graph.
            sink (int): index the sink node of the graph.
            parent (array): vector for keeping track of the parents of visited nodes.
        Attributes:
            visit (array): vector for keeping track of visited nodes,
            queue (list): the queue of the nodes needed to be visited,
            parent (array): vector for keeping track of the parents of visited nodes.
        Returns:
            (bool): A True/False indicating the presence or absence of a path. 
            It also updates the parent list with the information necessary to reconstruct the path.
  
        '''
        # se inicia el vector de visit en 0 de acuerdo al nuemero de nodos
        cdef array.array v = array.array('i', [0]*(self.N))
        cdef signed int[:] visit  = v 
        
        # se inicia el vector queue vacio     
        queue=[]    
        # se agerga al vector queue el valor  de source y 
        # mediante source se asigna al primer espacio del vector visit en true 
        #para comenzar nuestra busqueda del path
        queue.append(source)              
        visit[source]=1               

        while queue:

            # se extrae siempre el primer valor del queue y se asigna a u
            u = queue.pop(0)
            
            # Se requiere tanto el index como el valor del nodo que sera padre
            for index, value in enumerate(self.graph[u]): 
                if visit[index]== 0 and value > 0:
                    queue.append(index)
                    visit[index] = 1
                    parent[index]=u

        # Se crea un check ternario para regresar True 
        # si el camino sido recorrido completamente False de lo contrario
        return True if visit[sink]==1 else False
  
    # metodo para ejecutar algoritmo de ford fulkerson
    def ford_fulkerson(self):
        '''
        Runs the Ford Fulkerson method, keeping and updating the residual graph 
        and running over the bfs function in all the nodes.
        Args:
            source (int): index the source node of the graph.
            sink (int): index the sink node of the graph.
        Attributes:
            graph (numpy array):matrix the residual graph,
            path_flow (float): we need to calculate the min flow of the selected path,
            parent (array): vector for keeping track of the parents of visited nodes.
        Returns:
            Print calculated maximum flow of the graph.
            
        '''

        # se inicia parent en -1 de acuerdo a la cantidad de nodos
        cdef array.array p = array.array('i', [-1]*(self.N))
        cdef signed int[:] parent = p

        cdef unsigned int v, j
        cdef signed int u
            
        while self.busq_anchura(self.source,self.sink,parent):
            # se inicia pathflow en inf float 
            # que contendra el minimo flujo del path seleccionado
            path_flow = float('inf')     
            j = self.sink                    
                
            while j != self.source:
                # se calcula el minimo de todo el path 
                path_flow=min(path_flow, self.graph[parent[j]][j])
                # se asigna el valor del nodo padre  
                j=parent[j]

            
            # se actualiza los valores residuales de los edges en self.residualgraph
            v = self.sink
            while v != self.source:
                u=parent[v]
                self.graph[u][v] -= path_flow                   
                self.graph[v][u] += path_flow
                v=parent[v]


            # se agrega el path_flow para calcular el maximo
            self.maximum_flow += path_flow  
            
        return print("maximun flow is",self.maximum_flow)
    
    def get_maximumflow(self):
        '''
        Get the Maximun flow of the graph, calculate tu algorithom
        Returns:
            maximum_flow (int): calculated maximum flow of the graph.
        '''
        return self.maximum_flow

    # metodo para obtener información del grafo
    def infoMF(self):
        '''
        Obtains the information of the graph, regarding to the number of nodes and arcs
        Args:
            grapf (matrix): defines the graph from a matrix.
        Returns:
            Printed info of the graph.
            
        '''
        # Función para contar nodos y ramas
        cdef unsigned int i=0, m=0, rama=0
        
        arreglo = self.graph
        for i in range(len(arreglo)):
            for m in range(len(arreglo[i])):
                if arreglo[i][m] == 0:
                    pass
                else:
                    rama = rama + 1
        print("nodos",len(arreglo))
        print("ramas",rama)
        

Para esta iteración utilizamos numpy como orquestador y constructor de los grafos, se declaran todos los tipos de las variables y constantes a utilizar con el objetivo de que el compilado interprete estos y baje el tiempo de asignacion de memoria y performance de los procesos adicinal se puede observar que las lienas amarillas disminuyeron considerablemente. 

Se creo una nueva funcion get_maximumflow() que devuelve maximun flow calculado como variable tipo int.

Se carga nuestro modulo compilado 

### Prueba de la reimplementación

In [41]:
%%file script/setup_4.py
from distutils.core import setup
from Cython.Build import cythonize

setup(ext_modules = cythonize("script/mfaeiu4.pyx", 
                              compiler_directives={'language_level' : 3}))

Overwriting script/setup_4.py


### Compilación desde línea de comandos

In [42]:
%%bash
python3 script/setup_4.py build_ext --inplace

Compiling script/mfaeiu4.pyx because it changed.
[1/1] Cythonizing script/mfaeiu4.pyx
running build_ext
building 'mfaeiu4' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.8 -c script/mfaeiu4.c -o build/temp.linux-x86_64-3.8/script/mfaeiu4.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.8/script/mfaeiu4.o -o /datos/practica-2-segunda-parte-EddOselotl/notebooks_apoyo/mfaeiu4.cpython-38-x86_64-linux-gnu.so


In [46]:
from mfaeiu4 import MaxFlowAeiu

In [47]:
url_d = "https://raw.githubusercontent.com/optimizacion-2-2022-gh-classroom/practica-2-primera-parte-urieluard/main/BD/d.csv"
d = pd.read_csv(url_d,header=None)
arreglo = d.values.tolist()

In [48]:
start_time = time.time()
MF=MaxFlowAeiu(arreglo)
MF.ford_fulkerson()
print("The maximum flow in this network is {}".format(MF.get_maximumflow()))
end_time = time.time()

maximun flow is 1480
The maximum flow in this network is 1480


In [49]:
secs_4 = end_time-start_time
print("El algoritmo tomó",secs_4,"segundos" )

El algoritmo tomó 0.019102096557617188 segundos


In [50]:
print("Redujimos el tiempo en: {} segundos".format(secs_3 - secs_4))

Redujimos el tiempo en: 0.4579200744628906 segundos


In [51]:
print("Redujimos el tiempo en: {} segundos con respecto al original".format(secs_1 - secs_4))
print("Redujimos el tiempo en: {} segundos con respecto a la segunda reimplementación".format(secs_2 - secs_4))
print("Redujimos el tiempo en: {} segundos con respecto a la tercera reimplementación".format(secs_3 - secs_4))

Redujimos el tiempo en: 0.4035828113555908 segundos con respecto al original
Redujimos el tiempo en: 0.3175187110900879 segundos con respecto a la segunda reimplementación
Redujimos el tiempo en: 0.4579200744628906 segundos con respecto a la tercera reimplementación


In [52]:
print("La reimplementación es {} veces más rápida que nuestro código original compilado a C.".format(secs_1 / secs_4))

La reimplementación es 22.127670993509735 veces más rápida que nuestro código original compilado a C.


## Referencias
[1] [Capítulo 5.3 del Libro de Optimización](https://itam-ds.github.io/analisis-numerico-computo-cientifico/5.optimizacion_de_codigo/5.3/Compilacion_a_C.html#cython)